In [1]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

Dataset URL: https://www.kaggle.com/datasets/vivek468/beginner-chest-xray-image-classification
License(s): other
 99% 2.06G/2.08G [00:20<00:00, 174MB/s]
100% 2.08G/2.08G [00:20<00:00, 110MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader



In [ ]:
import zipfile

# ZIP dosyasının yolu
zip_file_path = '/content/covid19-radiography-database.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

data_dir = '/content/COVID-19_Radiography_Dataset'

train_dir = '/content/TrainTemp'
test_dir = '/content/Test'

classes = ['COVID', 'Normal', 'Viral Pneumonia']

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Her sınıf için eğitim ve test dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Her sınıf için görsel dosyalarını eğitim ve test dizinlerine kopyala
for cls in classes:
    cls_dir = os.path.join(data_dir, cls, 'images')
    images = os.listdir(cls_dir)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_dir, cls, img)
        shutil.copy(src, dst)

    for img in test_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(test_dir, cls, img)
        shutil.copy(src, dst)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

train_dir = '/content/TrainTemp'

train_data_dir = '/content/Train'
validation_data_dir = '/content/Validation'
test_data_dir = '/content/Test'

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(validation_data_dir, exist_ok=True)

classes = ['COVID', 'Normal', 'Viral Pneumonia']

# Her sınıf için eğitim ve doğrulama dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_data_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(validation_data_dir, cls), exist_ok=True)

# Her sınıf için eğitim verisini eğitim ve doğrulama setlerine ayır
for cls in classes:
    cls_dir = os.path.join(train_dir, cls)
    train_images = os.listdir(cls_dir)
    train_images, validation_images = train_test_split(train_images, test_size=0.125, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_data_dir, cls, img)
        shutil.copy(src, dst)

    for img in validation_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(validation_data_dir, cls, img)
        shutil.copy(src, dst)


import shutil

shutil.rmtree("/content/TrainTemp")

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



# Veri dönüşümleri
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'Train': datasets.ImageFolder(root=train_data_dir, transform=data_transforms['Train']),
    'Validation': datasets.ImageFolder(root=validation_data_dir, transform=data_transforms['Validation']),
    'Test': datasets.ImageFolder(root=test_data_dir, transform=data_transforms['Test'])
}

dataloaders = {
    'Train': DataLoader(image_datasets['Train'], batch_size=32, shuffle=True),
    'Validation': DataLoader(image_datasets['Validation'], batch_size=32, shuffle=False),
    'Test': DataLoader(image_datasets['Test'], batch_size=32, shuffle=False)
}


In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim


# Modeli yükle
densenet121 = models.densenet121(pretrained=True)

# Son fully connected layer'ı değiştirme
num_ftrs = densenet121.classifier.in_features
densenet121.classifier = nn.Linear(num_ftrs, len(classes))  # Üç sınıf için çıkış

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
densenet121 = densenet121.to(device)

# Optimizasyon algoritması ve kayıp fonksiyonu
optimizer = optim.SGD(densenet121.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()



In [5]:
import torch.nn.functional as F
import torch.nn as nn
import copy
from tqdm import tqdm

# Eğitim döngüsü
epochs = 3
best_model_wts = copy.deepcopy(densenet121.state_dict())
best_acc = 0.0

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    for phase in ['Train', 'Validation']:
        if phase == 'Train':
            densenet121.train()
        else:
            densenet121.eval()

        running_loss = 0.0
        running_corrects = 0

        data_loader = tqdm(dataloaders[phase], desc=f'{phase} Phase', unit="batch", leave=True)
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'Train'):
                outputs = densenet121(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'Train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            data_loader.set_postfix({'loss': loss.item()})

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'Validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(densenet121.state_dict())

    print()

# En iyi model ağırlıklarını yükleme
densenet121.load_state_dict(best_model_wts)


Epoch 1/1: 100%|██████████| 1829/1829 [03:48<00:00,  8.02batch/s, Loss=0.00263]

Epoch 1/1, Loss: 0.0117


In [6]:
# Doğruluk ve sınıflandırma metriklerini hesaplama
correct = 0
total = 0
confusion_matrix = np.zeros((len(classes), len(classes)))

with torch.no_grad():
    for inputs, labels in dataloaders['Test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = densenet121(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Confusion matrix'i güncelle
        for t, p in zip(labels.view(-1), predicted.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1

# Doğruluk hesapla
accuracy = correct / total

# Her sınıf için kesinlik, duyarlılık ve F1 skorlarını hesapla
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
f1_score = 2 * precision * recall / (precision + recall)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")
print("Confusion Matrix:")
print(confusion_matrix)


Test Accuracy: 0.9677
Precision: 1.0000
Recall: 0.9482
F1-Score: 0.9734
TP: 366
FP: 0
FN: 20
TN: 234


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Heatmap oluştur
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt="g", cmap="rocket_r", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
